# Método de la Potencia

En muchas aplicaciones del mundo real de la ciencia y la ingeniería, se requiere encontrar numéricamente el valor Eigen más grande o dominante y el Eigenvector correspondiente. Existen diferentes como el método de la potencia que sigue un enfoque iterativo y conforma un algoritmo simple.

El siguiente algoritmo busca desarrollar los pasos necesario para el método de la potencia y así después pues ser implementado como otro método alternativo a los demás que se tienen en el proyecto:

Sea **A** una matriz cuadrada de orden **n**, es decir, **An x n**. El Método de Potencia comienza con una aproximación inicial al Eigenvector correspondiente al valor Eigen más grande de tamaño **n x 1**. Sea esta aproximación inicial **Xn x 1**.

Después de la suposición inicial, calculamos **AX**, es decir, el producto de la matriz **A** y **X**. Del producto de **AX** dividimos cada elemento por el elemento más grande (por magnitud) y los expresamos como **λ1X1**. El valor obtenido de **λ1** y **X1** es el siguiente valor aproximado mejor del valor Eigen más grande y el Eigenvector correspondiente.

Del mismo modo, para el siguiente paso, multiplicamos **A** por **X1**. Del producto de **AX1** dividimos cada elemento por el elemento más grande (por magnitud) y los expresamos como **λ2X2**. El valor obtenido de **λ2** y **X2** es el siguiente valor aproximado mejor del valor Eigen más grande y el Eigenvector correspondiente.

Se repite lo anterior iterativamente hasta obtener el valor Eigen más grande o dominante y el Eigenvector correspondiente con la precisión deseada.

El algoritmo sería programado de la siguiente forma:

 
  1. Se lee el orden de la matriz (n) y el error tolerable (e)
 
  2. Se lee la matriz A de tamaño n x n
 
  3. Se crea un vector inicial X de tamaño n x 1
 
  4. Inicializar: Lambda_Old = 1
 
  5. Multiplicar: X_NEW = A * X
 
  6. Reemplace X por X_NEW
 
  7. Encuentra el elemento más grande (Lamda_New) por magnitud desde X_NEW
 
  8. Normalizar o dividir X entre Lamda_Nuevo
 
  9. Mostrar Lamda_New y X

  10. Si | Lambda_Old - Lamda_New | > e entonces
      establecer Lambda_Old = Lamda_New y volver a ejecutar desde el paso 6, de lo contrario detener el algoritmo.


Basado en las notas y ejemplo de:

Power Method Algorithm for Finding Dominant Eigen Value and Eigen Vector. (n.d.). Retrieved May 23, 2020, from https://www.codesansar.com/numerical-methods/power-method-algorithm-for-finding-dominant-eigen-value-and-eigen-vector.htm

Función de R para generar ejemplos:

Fox, J., Chalmers, P., Monette, G., & Sanchez, G. (2020, April 14). powerMethod: Power Method for Eigenvectors in matlib: Matrix Functions for Teaching and Learning Linear Algebra and Multivariate Statistics. Retrieved from https://rdrr.io/cran/matlib/man/powerMethod.html

Función que integra PCA:

Dan, D. J. (n.d.). dianejdan/Power-Method-PCA. Retrieved May 27, 2020, from https://github.com/dianejdan/Power-Method-PCA/blob/master/power-pca.py



In [19]:
import sys
import numpy as np

#Se genera una matriz como el ejemplo de las notas
np.random.seed(2020)
mpoints=200
ncols = 3
X = (np.random.rand(ncols,ncols)@np.random.normal(0,1,(ncols,mpoints))).T
m,n = X.shape

### Se centran los datos y se calcula la matriz de covarianzas
X_centered = X-np.mean(X, axis=0)
cov = np.dot(X_centered.T, X_centered)/X_centered.shape[0]

print('Matriz de Covarianzas')
print(cov)

### Método de la potencia para PCA
eps = 1e-06 #parámetro de convergencia para finalizar las iteraciones, se deja en 1e-06 conforme a la documentación de R
dif=1

#Inicializar: Lambda_Old = 1
#Multiplicar: X_NEW = A * X
#Reemplace X por X_NEW
#Encuentra el elemento más grande (Lamda_New) por magnitud desde X_NEW
#Normalizar o dividir X entre Lamda_Nuevo
#Mostrar Lamda_New y X
#Si | Lambda_Old - Lamda_New | > e entonces establecer Lambda_Old = Lamda_New y volver a ejecutar desde el paso 6, de lo contrario detener el algoritmo.

#Se crean dos matrices con 1's
w = np.ones(X.shape[1])
w_new = np.ones(X.shape[1])


while (dif > eps): #Iteramos hasta llegar al parámetro de convergencia
    w = w_new
    w_tmp = np.dot(cov, w)  #se multiplica la matriz de covarianzas por w.
    ind = np.abs(w_tmp).argmax() #Calculate the absolute value element-wise. #Returns the indices of the maximum values along an axis.
    w_new = w_tmp/w_tmp[ind]
    dif = np.dot(w_new-w, w_new-w)

eig_val = np.dot(np.dot(cov, w_new), w_new)/np.dot(w_new, w_new)
eig_vec = w_new/np.sqrt(np.dot(w_new, w_new))

eig_val_true, eig_vec_true = np.linalg.eig(cov)

print('El valor Eigen más grande es' % eig_val)
print('Su eigenvector correspondiente es')
print(eig_vec)


### Calcula la proporción de la varianza explicada por los dos primeros PCs
X_proj = np.dot(X_centered, eig_vec_true[:,[0,1]])
var1 = np.var(X_proj[:,0])
var2 = np.var(X_proj[:,1])
print('Suma de la varianza de los eigenvectores' % (var1+var2))

###  proyecta los datos que apuntan al espacio abarcado por las PC que explican más del 90% de la varianza
def calPCA(sigma, comp):
    eig_vals, eig_vecs = np.linalg.eig(sigma)
    return {'eig_val': eig_vals[comp-1], 'eig_vec': eig_vecs[:,comp-1]}


total_var = 0
for i in range(X.shape[1]):
    total_var = total_var + calPCA(cov, i+1)['eig_val']
    
prop = 0
comp = 1 
cur_var = 0
comp_vecs = np.zeros([X.shape[1], X.shape[1]])   
while (prop < 0.9):
    cur_var = cur_var + calPCA(cov, comp)['eig_val']
    prop = cur_var/total_var
    comp_vecs[:,comp-1] = calPCA(cov, comp)['eig_vec']
    comp = comp+1

X_new = np.dot(X, comp_vecs[:,range(comp-1)]) 
print('Las nuevas coordenadas de los primeros 10 puntos de datos son')
print(X_new[0:10,])

Matriz de Covarianzas
[[1.87079983 0.6268753  0.84175899]
 [0.6268753  0.21691674 0.31579903]
 [0.84175899 0.31579903 0.77059722]]
The largest eigenvalue I calculated is 2.54

Its correspondin eigenvector is
[0.84313138 0.28933077 0.45322973]


The largest eigenvalue calculated by np.linalg.eig is 2.54

Its correponding eigenvector is
[-0.84315718 -0.28933414 -0.45317956]


sum of variance of projection on the first two eigenvectors is 2.85

The new coordinates of the first 10 data points are
[[-0.40474378  0.17337183]
 [ 0.80744656 -0.03100271]
 [-1.52757559  0.02667735]
 [ 0.34605851 -0.58106658]
 [ 1.70443306  0.26647624]
 [ 2.81431215 -0.03352812]
 [ 0.87392525 -0.15708527]
 [-0.4238356  -0.24135029]
 [ 2.21278866  0.37628692]
 [ 0.10525245 -0.16856843]]


In [0]:

import numpy as np
from pylab import *

#Se genera una matriz como el ejemplo de las notas
np.random.seed(2020)
mpoints=200
ncols = 3
X = (np.random.rand(ncols,ncols)@np.random.normal(0,1,(ncols,mpoints))).T
m,n = X.shape

def power_iteration(A, num_simulations: int):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k
    
pw= power_iteration(X, 10)